In [273]:
import numpy as np
import pandas as pd
import os
import datetime as dt
from pykrx import stock


from dateutil import relativedelta

In [310]:
'''뒤지게 오래걸림 import할때'''

kospi200 = stock.get_index_ohlcv('20070901', '20221130', '1028')['종가']
file_path = os.getcwd() + '/Data/'

df_close   = pd.read_excel(file_path+'Call_data_close.pkl', sheet_name='종가')
df_base    = pd.read_excel(file_path+'Call_data_base.pkl', sheet_name='기준가')
df_volume  = pd.read_excel(file_path+'Call_data_volume.pkl', sheet_name='거래량')
df_imvol   = pd.read_excel(file_path+'Call_data_imvol.pkl', sheet_name='내재변동성')
df_open    = pd.read_excel(file_path+'Call_data_open.pkl', sheet_name='미결제약정')

data_list = [df_close, df_base, df_volume, df_imvol, df_open]


In [311]:
df_close

,Unnamed: 0,Last Refresh: 2022-11-30 22:21:30,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 8451,Unnamed: 8452,Unnamed: 8453,Unnamed: 8454,Unnamed: 8455,Unnamed: 8456,Unnamed: 8457,Unnamed: 8458,Unnamed: 8459,Unnamed: 8460
0,Calendar Basis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio,,Default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Item,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Frequency,일간,KRW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Non-Trading Day,Exclusive,Asc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,2022-11-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3798,2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3799,2022-11-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3800,2022-11-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [275]:
def ch (name):
    cptype = name[7:8]
    maturity = name[9:15]
    strike = name[16:]

    return cptype+maturity+strike

def op_chain_df (cpflag, maturity, strike, data_list, s0=None):
    '''data_list 순서 (종가/기준가/거래량/내재변동성/미결제약정)'''

    name = cpflag + str(maturity) + str(strike)

    reg = pd.DataFrame()

    for idx, feature in enumerate(['종가', '기준가', '거래량', '내재변동성', '미결제약정']):
        df = data_list[idx]

        tmp = df.iloc[8:]
        tmp = tmp.set_axis(tmp.iloc[0], axis=1).drop(index=[8,9,10,11,12])

        tmp = tmp.rename(columns={tmp.columns[0] : 'Date'}).set_index('Date')
        tmp.index = pd.to_datetime(tmp.index)
        tmp.columns = [ch(x) for x in tmp.columns]

        reg = pd.concat([reg, tmp[name]], axis=1)

    reg.index = tmp.index
    reg.columns = ['종가', '기준가', '거래량', '내재변동성', '미결제약정']

    if s0 is None:
        return reg.dropna()
    else:
        reg.dropna(inplace=True)
        reg['s0'] = s0.loc[reg.index]
        return reg


def option_date_creator (week, date):
    '''마지막 만기 거래일이 나옴'''
    base_date = dt.datetime.strptime(date, '%Y-%m')
    option_date = base_date + relativedelta.relativedelta(weekday=3) + dt.timedelta(weeks=week)
    option_date = option_date.strftime('%Y-%m-%d')

    return option_date

## 데이터내부의 K의 Boundary와 만기일

In [276]:
ticker_list = [ch(x) for x in df_close.iloc[8].values][1:]
maturity_date = list(pd.Series([str(x[1:7]) for x in ticker_list]).unique())

upper = []
lower = []

for date in maturity_date:
    buffer = []
    for tick in ticker_list:
        if tick[1:7] == date:
            buffer.append(tick[7:])
        else:
            pass


    upper.append(buffer[np.argmax(buffer)])
    lower.append(buffer[np.argmin(buffer)])

reg = pd.DataFrame(data=[upper, lower], columns=maturity_date).T.rename(columns={0:'Upper', 1:'Lower'})
reg
reg['last_trade_date'] = [option_date_creator(1, f'{x[:4]}-{x[4:]}') for x in reg.index]
reg

,Upper,Lower,last_trade_date
200709,270.0,165.0,2007-09-13
200710,275.0,192.5,2007-10-11
200711,277.5,192.5,2007-11-08
200712,277.5,195.0,2007-12-13
200801,277.5,210.0,2008-01-10
...,...,...,...
202207,515.0,222.5,2022-07-14
202208,490.0,222.5,2022-08-11
202209,520.0,220.0,2022-09-08
202210,480.0,180.0,2022-10-13


## 옵션 뽑기 (Data_list에 5개의 df 설정해주기)

In [277]:
test = op_chain_df('C', 200712, 277.5, data_list)
test

,종가,기준가,거래량,내재변동성,미결제약정
Date,,,,,
2007-10-12,4.85,6.2,2165,0.264,1591
2007-10-15,4.5,4.85,3752,0.259,4078
2007-10-16,3.2,4.5,2872,0.253,4941
2007-10-17,2.43,3.2,5292,0.251,7059
2007-10-18,2.68,2.43,4227,0.248,9073
2007-10-19,1.8,2.68,1669,0.244,9311
2007-10-22,1.07,1.8,3859,0.268,11105
2007-10-23,1.45,1.07,4644,0.262,13356
2007-10-24,1.25,1.45,15102,0.264,15261


In [278]:
test = op_chain_df('C', 200712, 277.5, data_list, kospi200)
test

,종가,기준가,거래량,내재변동성,미결제약정,s0
Date,,,,,,
2007-10-12,4.85,6.2,2165,0.264,1591,257.31
2007-10-15,4.5,4.85,3752,0.259,4078,257.68
2007-10-16,3.2,4.5,2872,0.253,4941,253.43
2007-10-17,2.43,3.2,5292,0.251,7059,250.48
2007-10-18,2.68,2.43,4227,0.248,9073,252.60
2007-10-19,1.8,2.68,1669,0.244,9311,248.35
2007-10-22,1.07,1.8,3859,0.268,11105,239.94
2007-10-23,1.45,1.07,4644,0.262,13356,244.90
2007-10-24,1.25,1.45,15102,0.264,15261,243.31
